In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Note: this cell can only be run once, since we're changing the directory.
import sys
import os

path = "D:/testings/Python/TestingPython/"
if path not in sys.path:
    sys.path.append(path)

# otfusion_path = "D:/testings/Python/TestingPython/ModelFusion/otfusion/"
# if otfusion_path not in sys.path:
#     sys.path.append(otfusion_path)
os.chdir("../")
os.getcwd()

'D:\\testings\\Python\\TestingPython\\ModelFusion'

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from ModelFusion.wasserstein_ensemble import (
    geometric_ensembling_modularized,
    get_wassersteinized_layers_modularized,
    get_acts_wassersteinized_layers_modularized
)
from ModelFusion.helpers.utils import (
    load_yml_file
)
from ModelFusion.helpers.load_model import reload_model
from ModelFusion.helpers.model2graph import model2graph_wrapper, export_as_onnx
from ModelFusion.wasserstein_ensemble_ViT import ViTFuser

## Read args

In [4]:
fusion_config_path = "configs/fusion_configs.yml"
config = load_yml_file(fusion_config_path)

## Reload Models

In [5]:
model_path_1 = "./clf_logs/2022_11_13_02_20_43_443389/"
model_path_2 = "./clf_logs/2022_11_30_08_08_57_761624/"
model_name = "ViT"
model_1 = reload_model(model_name, model_path_1)
model_2 = reload_model(model_name, model_path_2)

In [6]:
model_1

ViT(
  (patch_embedding): PatchEmbeddingBlock(
    (patch_embeddings): Conv2d(3, 768, kernel_size=(4, 4), stride=(4, 4), bias=False)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (blocks): ModuleList(
    (0): TransformerBlock(
      (mlp): MLPBlock(
        (linear1): Linear(in_features=768, out_features=3072, bias=False)
        (linear2): Linear(in_features=3072, out_features=768, bias=False)
        (fn): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): SABlock(
        (out_proj): Linear(in_features=768, out_features=768, bias=False)
        (qkv): Linear(in_features=768, out_features=2304, bias=False)
        (input_rearrange): Rearrange('b h (qkv l d) -> qkv b l h d', qkv=3, l=12)
        (out_rearrange): Rearrange('b h l d -> b l (h d)')
        (drop_output): Dropout(p=0.0, inplace=False)
        (drop_weights): Dropout(p=0.0, inpl

In [7]:
# L = 768
# ln = nn.LayerNorm(L)
# list(ln.named_parameters())

In [8]:
for name, param in model_1.named_parameters():
    print(f"{name}: {param.shape}, {param.ndim}")
#     if "norm" in name:
#         print(param[:10])

cls_token: torch.Size([1, 1, 768]), 3
patch_embedding.position_embeddings: torch.Size([1, 64, 768]), 3
patch_embedding.patch_embeddings.weight: torch.Size([768, 3, 4, 4]), 4
blocks.0.mlp.linear1.weight: torch.Size([3072, 768]), 2
blocks.0.mlp.linear2.weight: torch.Size([768, 3072]), 2
blocks.0.norm1.weight: torch.Size([768]), 1
blocks.0.norm1.bias: torch.Size([768]), 1
blocks.0.attn.out_proj.weight: torch.Size([768, 768]), 2
blocks.0.attn.qkv.weight: torch.Size([2304, 768]), 2
blocks.0.norm2.weight: torch.Size([768]), 1
blocks.0.norm2.bias: torch.Size([768]), 1
blocks.1.mlp.linear1.weight: torch.Size([3072, 768]), 2
blocks.1.mlp.linear2.weight: torch.Size([768, 3072]), 2
blocks.1.norm1.weight: torch.Size([768]), 1
blocks.1.norm1.bias: torch.Size([768]), 1
blocks.1.attn.out_proj.weight: torch.Size([768, 768]), 2
blocks.1.attn.qkv.weight: torch.Size([2304, 768]), 2
blocks.1.norm2.weight: torch.Size([768]), 1
blocks.1.norm2.bias: torch.Size([768]), 1
blocks.2.mlp.linear1.weight: torch.Siz

## Fuse non-fc layers

In [9]:
fuser = ViTFuser([model_1, model_2], config)

In [10]:
aligned_weights_dict = fuser()

Previous layer shape is  None
Processing the coordinates to form ground_metric
returns a uniform measure of cardinality:  768
returns a uniform measure of cardinality:  768
the transport map is  tensor([[3.3085e-07, 3.8550e-11, 1.1301e-17,  ..., 3.8042e-11, 2.7480e-11,
         2.4980e-11],
        [3.6191e-11, 1.1701e-12, 3.8227e-09,  ..., 1.0288e-11, 1.1302e-08,
         5.9800e-14],
        [3.8945e-14, 2.8679e-16, 4.5270e-12,  ..., 1.9598e-15, 1.9551e-14,
         6.6230e-13],
        ...,
        [4.2885e-11, 3.5610e-12, 1.3808e-13,  ..., 4.0169e-10, 7.8354e-11,
         4.3993e-08],
        [2.0361e-09, 4.8177e-04, 5.7465e-15,  ..., 9.1433e-10, 9.5169e-08,
         6.2155e-12],
        [1.1476e-12, 1.3175e-16, 1.2533e-14,  ..., 3.0473e-11, 4.7878e-05,
         1.7008e-10]], device='cuda:0')
Ratio of trace to the matrix sum:  tensor(0.0006, device='cuda:0')
Here, trace is 0.0005825987318530679 and matrix sum is 1.0 
-----------------------------------------------------------------